In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import os
import matplotlib.pyplot as plt
import numpy as np
from torch.autograd import Variable

# ============================= #
# you can define your own model #
# ============================= #

In [3]:
# class myNet(nn.Module):
#     #define the layers
#     def __init__(self):
#         super(myNet, self).__init__()

#     def forward(self, x):
#         return x

class LeNet(nn.Module):
    #define the layers
    def __init__(self):
        super(LeNet, self).__init__()
        print('Building model...')
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 8 * 2809, 120)
        self.fc2 = nn.Linear(120, 84)
        #self.fc3 = nn.Linear(84, 10)
        self.fc3 = nn.Linear(84, 2)
        self.relu = nn.ReLU()
        
    # connect these layers
    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 16 * 8 * 2809)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [6]:
class StockPrediction(object):
    """docstring for StockPrediction"""
    def __init__(self, in_path, epoch, batch_size, lr):
        super(StockPrediction, self).__init__()
        self.in_path = in_path
        self.epoch = epoch
        self.batch_size = batch_size
        self.lr = lr

        self.classes = ['not_rise', 'rise']

        self.checkdevice()
        self.prepareData()
        self.getModel()
        self.train_acc = self.train()
        self.saveModel()
        self.test()

        #self.showWeights()

    def checkdevice(self):
        # To determine if your system supports CUDA
        print("Check devices...")
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print("Current device:", self.device)

        # Also can print your current GPU id, and the number of GPUs you can use.
        print("Our selected device:", torch.cuda.current_device())
        print(torch.cuda.device_count(), "GPUs is available")
        return

    def prepareData(self):
        # The output of torchvision datasets are PILImage images of range [0, 1]
        # We transform them to Tensor type, ToTensoer:歸一化至[0, 1]
        # And normalize the data
        # Be sure you do same normalization for your train and test data
        print('Preparing dataset...')

        # The transform function for train data
        transform_train = transforms.Compose([
            #transforms.RandomCrop(128, padding=4),
            #transforms.RandomHorizontalFlip(),
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
            # you can apply more augment function
            # [document]: https://pytorch.org/docs/stable/torchvision/transforms.html
        ])

        # The transform function for test data
        transform_test = transforms.Compose([
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
        ])
        
        
        # The transform function for valid
        transform_valid = transforms.Compose([
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
        ])


        # TODO
        self.trainset = torchvision.datasets.ImageFolder("./classification/train/", transform=transform_train)
        self.testset = torchvision.datasets.ImageFolder("./classification/test/", transform=transform_test)
        self.validset = torchvision.datasets.ImageFolder("./classification/valid/", transform=transform_valid)

        
        self.trainloader = torch.utils.data.DataLoader(self.trainset, batch_size=self.batch_size, shuffle=True)
        self.testloader = torch.utils.data.DataLoader(self.testset, batch_size=self.batch_size, shuffle=False)
        # you can also split validation set
        self.validloader = torch.utils.data.DataLoader(self.validset, batch_size=self.batch_size, shuffle=True)
        return

    def getModel(self):
        # Build a Convolution Neural Network
        self.net = torch.hub.load('pytorch/vision:v0.6.0', 'resnext101_32x8d', pretrained=True)
        
        #####未知區域#####
        for count, child in enumerate(self.net.children()):
            for param in child.parameters():
                param.requires_grad = False
        
        num_ftrs = self.net.fc.in_features
        self.net.fc = nn.Linear(num_ftrs, 2)
        
        print(self.net)

        # Define loss function and optimizer，SGD(momentum=0.9)
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.SGD(self.net.parameters(), lr=self.lr, momentum=0.9)
        return
        
    def train(self):
        print('Training model...')
        # Change all model tensor into cuda type
        # something like weight & bias are the tensor 
        self.net = self.net.to(self.device) 
        
        # Set the model in training mode
        # because some function like: dropout, batchnorm...etc, will have 
        # different behaviors in training/evaluation mode
        # [document]: https://pytorch.org/docs/stable/nn.html#torch.nn.Module.train
        self.net.train()
        for e in range(self.epoch):  # loop over the dataset multiple times
            train_running_loss = 0.0
            valid_running_loss = 0.0
            train_correct = 0
            valid_correct = 0
            self.net.train()
            
            for i, (inputs, labels) in enumerate(self.trainloader, 0):
                
                #change the type into cuda tensor 
                inputs = inputs.to(self.device) 
                labels = labels.to(self.device) 

                # zero the parameter gradients (梯度归零)
                self.optimizer.zero_grad()

                # forward + backward + optimize
                outputs = self.net(inputs)
                # select the class with highest probability
                _, pred = outputs.max(1)
                # if the model predicts the same results as the true
                # label, then the correct counter will plus 1
                train_correct += pred.eq(labels).sum().item()  
                
                loss = self.criterion(outputs, labels)
                
                #方向传播
                loss.backward()
                #更新参数
                self.optimizer.step()

                # print statistics
                train_running_loss += loss.item()
                
            #   if i % 100 == 99:    # print every 200 mini-batches
            #       print('[%d, %5d] loss: %.3f' %
            #             (e + 1, i + 1, running_loss / 100))
            #       running_loss = 0.0
            
            #驗證集測試並印出結果
            self.net.eval()
            
            for i, (inputs, labels) in enumerate(self.validloader, 0):                
                inputs = inputs.to(self.device) 
                labels = labels.to(self.device) 

                # forward + backward + optimize
                outputs = self.net(inputs)
                # select the class with highest probability
                _, pred = outputs.max(1)
                
                valid_correct += pred.eq(labels).sum().item()
                loss = self.criterion(outputs, labels)
                valid_running_loss += loss.item()

            train_average_loss = train_running_loss / len(self.trainloader.dataset)
            valid_average_loss = valid_running_loss / len(self.validloader.dataset)
           
            print("%d epoch, train_loss: %.4f, valid_loss :%.4f" % (e+1, train_average_loss, valid_average_loss))
            print('%d epoch, training accuracy: %.4f, valid accuracy: %.4f' % (e+1, 100.*train_correct/len(self.trainset), 100*valid_correct/len(self.validset)))
            
        print('Finished Training')
        return 100.*train_correct/len(self.trainset)

    def test(self):
        print('==> Testing model..')
        # Change model to cuda tensor
        # or it will raise when images and labels are all cuda tensor type
        self.net = self.net.to(self.device)

        # Set the model in evaluation mode
        # [document]: https://pytorch.org/docs/stable/nn.html#torch.nn.Module.eval 
        #让model变成测试模式，这主要是对dropout和batch normalization的操作在训练和测试的时候是不一样的
        self.net.eval()

        correct = 0
        running_loss = 0.0
        iter_count = 0
        class_correct = [0 for i in range(len(self.classes))]
        class_total = [0 for i in range(len(self.classes))]
        with torch.no_grad(): # no need to keep the gradient for backpropagation
            for data in self.testloader:
                images, labels = data
                
                images = images.to(self.device) 
                labels = labels.to(self.device)
                
                outputs = self.net(images)
                _, pred = outputs.max(1)
                correct += pred.eq(labels).sum().item()
                c_eachlabel = pred.eq(labels).squeeze()
            
                loss = self.criterion(outputs, labels)
                iter_count += 1
                running_loss += loss.item()
                for i in range(len(labels)):
                    #cur_label:表示標籤的項目
                    cur_label = labels[i].item()
                    try:
                        #將標籤的項目為True的+1，以計算該標籤正確總數，c_eachlabel[i]:為第幾項標籤預測正確or錯誤
                        class_correct[cur_label] += c_eachlabel[i].item()
                    except:
                        print(class_correct[cur_label])
                        print(c_eachlabel[i].item())
                    class_total[cur_label] += 1

        print('Total accuracy is: {:4f}% and loss is: {:3.3f}'.format(100 * correct/len(self.testset), running_loss/iter_count))
        print('For each class in dataset:')
        for i in range(len(self.classes)):
            print('Accruacy for {:18s}: {:4.2f}%'.format(self.classes[i], 100 * class_correct[i]/class_total[i]))

    def saveModel(self):
        # After training , save the model first
        # You can saves only the model parameters or entire model
        # Some difference between the two is that entire model 
        # not only include parameters but also record hwo each 
        # layer is connected(forward method).
        # [document]: https://pytorch.org/docs/master/notes/serialization.html
        print('Saving model...')

        # only save model parameters
        torch.save(self.net.state_dict(), './weight.t7')

        # you also can store some log information
        state = {
            'net': self.net.state_dict(),
            'acc': self.train_acc,
            'epoch': self.epoch
        }
        torch.save(state, './weight.t7')

        # save entire model
        torch.save(self.net, './model.pt')
        return

    def loadModel(self, path):
        print('Loading model...')
        if path.split('.')[-1] == 't7':
            # If you just save the model parameters, you
            # need to redefine the model architecture, and
            # load the parameters into your model
            self.net = LeNet()
            checkpoint = torch.load(path)
            self.net.load_state_dict(checkpoint['net'])
        elif path.split('.')[-1] == 'pt':
            # If you save the entire model
            self.net = torch.load(path)
        return

    #def showWeights(self):
        # TODO
        #w_conv1 = self.net.conv1.weight.reshape(-1).detach().cpu().numpy()
        #w_conv2 = self.net.conv2.weight.reshape(-1).detach().cpu().numpy()
        #w_fc1 = self.net.fc1.weight.reshape(-1).detach().cpu().numpy()
        #w_fc2 = self.net.fc2.weight.reshape(-1).detach().cpu().numpy()
        #w_fc3 = self.net.fc3.weight.reshape(-1).detach().cpu().numpy()

        #plt.figure(figsize=(24, 6))
        #plt.subplot(1,5,1)
        #plt.title("conv1 weight")
        #plt.hist(w_conv1)

        #plt.subplot(1,5,2)
        #plt.title("conv2 weight")
        #plt.hist(w_conv2)

        #plt.subplot(1,5,3)
        #plt.title("fc1 weight")
        #plt.hist(w_fc1)

        #plt.subplot(1,5,4)
        #plt.title("fc2 weight")
        #plt.hist(w_fc2)

        #plt.subplot(1,5,5)
        #plt.title("fc3 weight")
        #plt.hist(w_fc3)

        #plt.savefig('weights.png')

In [9]:
if __name__ == '__main__':
    # you can adjust your hyperperamers(path, epoch, batch_size, learning rate)
    ocr = StockPrediction("../data/train_lab2/",10 ,16 ,0.01 )

Check devices...
Current device: cuda
Our selected device: 0
1 GPUs is available
Preparing dataset...


Using cache found in C:\Users\bight/.cache\torch\hub\pytorch_vision_v0.6.0


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1

1 epoch, train_loss: 0.0527, valid_loss :0.3192
1 epoch, training accuracy: 71.8571, valid accuracy: 21.7391
2 epoch, train_loss: 0.0623, valid_loss :0.4280
2 epoch, training accuracy: 78.2143, valid accuracy: 14.7826
3 epoch, train_loss: 0.0433, valid_loss :0.3575
3 epoch, training accuracy: 80.7857, valid accuracy: 16.5217
4 epoch, train_loss: 0.0368, valid_loss :0.3235
4 epoch, training accuracy: 83.1429, valid accuracy: 22.1739
5 epoch, train_loss: 0.0340, valid_loss :0.2998
5 epoch, training accuracy: 83.8571, valid accuracy: 25.6522
6 epoch, train_loss: 0.0407, valid_loss :0.4321
6 epoch, training accuracy: 83.2857, valid accuracy: 40.8696
7 epoch, train_loss: 0.0370, valid_loss :0.3259
7 epoch, training accuracy: 83.5000, valid accuracy: 23.9130
8 epoch, train_loss: 0.0512, valid_loss :0.4318
8 epoch, training accuracy: 81.6429, valid accuracy: 22.6087
9 epoch, train_loss: 0.0444, valid_loss :0.3837
9 epoch, training accuracy: 83.8571, valid accuracy: 25.2174
10 epoch, train_los

In [77]:
#load model
model = ocr.loadModel("./model_E100B8L0001.pt")

Loading model...


In [ ]:
#測試
